In [1]:
# 📌 Install dependencies (for Colab)
!pip install pathway bokeh panel --quiet

# 📁 Imports
import pathway as pw
import pandas as pd
import numpy as np
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
import panel as pn

pn.extension()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

KeyError: 'date'

# Step 1: Initial Setup and Dataset Loading

In [2]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv("/content/dataset.csv")

# Check column names
print("Column Names:\n", df.columns.tolist())

# Display first few rows
df.head()


Column Names:
 ['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude', 'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime']


/usr/local/lib/python3.11/dist-packages/google/colab/_dataframe_summarizer.py:57: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(column, errors="raise")


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


#  Step 2: Preprocessing the Data

In [3]:
# Combine date and time into one datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True)

# Drop redundant columns if not needed
df.drop(['LastUpdatedDate', 'LastUpdatedTime', 'ID'], axis=1, inplace=True)

# Sort by parking space and time
df = df.sort_values(by=['SystemCodeNumber', 'Timestamp']).reset_index(drop=True)

# Preview the cleaned dataset
df.head()


,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,Timestamp
0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,2016-10-04 07:59:00
1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,2016-10-04 08:25:00
2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,2016-10-04 08:59:00
3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,2016-10-04 09:32:00
4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,2016-10-04 09:59:00


# Step 3: Model 1 – Baseline Linear Pricing

In [4]:
# Set base price and alpha
BASE_PRICE = 10
ALPHA = 2.0  # Changeable

# Initialize a new column
df['Price_Model1'] = np.nan

# Group by parking lot
for lot, group in df.groupby('SystemCodeNumber'):
    price = BASE_PRICE
    indices = group.index.tolist()

    for i in indices:
        occupancy = df.loc[i, 'Occupancy']
        capacity = df.loc[i, 'Capacity']
        delta = ALPHA * (occupancy / capacity)

        # Assign price
        df.loc[i, 'Price_Model1'] = price

        # Update price for next iteration
        price = price + delta

# Round off prices
df['Price_Model1'] = df['Price_Model1'].round(2)

# View results
df[['SystemCodeNumber', 'Timestamp', 'Occupancy', 'Capacity', 'Price_Model1']].head(10)


,SystemCodeNumber,Timestamp,Occupancy,Capacity,Price_Model1
0,BHMBCCMKT01,2016-10-04 07:59:00,61,577,10.00
1,BHMBCCMKT01,2016-10-04 08:25:00,64,577,10.21
2,BHMBCCMKT01,2016-10-04 08:59:00,80,577,10.43
3,BHMBCCMKT01,2016-10-04 09:32:00,107,577,10.71
4,BHMBCCMKT01,2016-10-04 09:59:00,150,577,11.08
5,BHMBCCMKT01,2016-10-04 10:26:00,177,577,11.60
6,BHMBCCMKT01,2016-10-04 10:59:00,219,577,12.21
7,BHMBCCMKT01,2016-10-04 11:25:00,247,577,12.97
8,BHMBCCMKT01,2016-10-04 11:59:00,259,577,13.83
9,BHMBCCMKT01,2016-10-04 12:29:00,266,577,14.73


#Step 4: Model 2 – Demand-Based Dynamic Pricing

 Add encodings for categorical features:

In [9]:
# Vehicle weights
vehicle_weights = {
    'car': 1.0,
    'bike': 0.5,
    'truck': 1.5,
    'cycle': 0.3  # Add cycle
}

# Traffic level mapping
traffic_map = {
    'low': 0.2,
    'medium': 0.5,
    'average': 0.5,  # Treat 'average' same as 'medium'
    'high': 1.0
}

# Apply
df['VehicleWeight'] = df['VehicleType'].map(vehicle_weights)
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)


Compute raw demand:

In [10]:
# Recalculate demand
df['RawDemand'] = (
    alpha * (df['Occupancy'] / df['Capacity']) +
    beta * df['QueueLength'] -
    gamma * df['TrafficLevel'] +
    delta * df['IsSpecialDay'] +
    epsilon * df['VehicleWeight']
)

# Normalize demand per lot
df['NormalizedDemand'] = df.groupby('SystemCodeNumber')['RawDemand'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min() + 1e-6)
)

# Recompute Price_Model2
df['Price_Model2'] = BASE_PRICE * (1 + lambda_factor * df['NormalizedDemand'])
df['Price_Model2'] = df['Price_Model2'].clip(lower=0.5 * BASE_PRICE, upper=2.0 * BASE_PRICE)
df['Price_Model2'] = df['Price_Model2'].round(2)


Normalize demand per parking lot:

In [7]:
# Normalized demand per lot
df['NormalizedDemand'] = df.groupby('SystemCodeNumber')['RawDemand'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min() + 1e-6)
)


In [11]:
df[['SystemCodeNumber', 'Timestamp', 'Occupancy', 'QueueLength', 'VehicleType',
    'TrafficConditionNearby', 'IsSpecialDay', 'Price_Model2']].head(10)


,SystemCodeNumber,Timestamp,Occupancy,QueueLength,VehicleType,TrafficConditionNearby,IsSpecialDay,Price_Model2
0,BHMBCCMKT01,2016-10-04 07:59:00,61,1,car,low,0,11.61
1,BHMBCCMKT01,2016-10-04 08:25:00,64,1,car,low,0,11.62
2,BHMBCCMKT01,2016-10-04 08:59:00,80,2,car,low,0,12.62
3,BHMBCCMKT01,2016-10-04 09:32:00,107,2,car,low,0,12.67
4,BHMBCCMKT01,2016-10-04 09:59:00,150,2,bike,low,0,12.22
5,BHMBCCMKT01,2016-10-04 10:26:00,177,3,car,low,0,13.78
6,BHMBCCMKT01,2016-10-04 10:59:00,219,6,truck,high,0,16.73
7,BHMBCCMKT01,2016-10-04 11:25:00,247,5,car,average,0,15.64
8,BHMBCCMKT01,2016-10-04 11:59:00,259,5,cycle,average,0,14.91
9,BHMBCCMKT01,2016-10-04 12:29:00,266,8,bike,high,0,17.67


Compute Model 2 price:

In [12]:
# Lambda: how much price should vary with demand
lambda_factor = 1.2  # tunable

# Price = Base * (1 + λ * demand) bounded between 0.5x and 2x
df['Price_Model2'] = BASE_PRICE * (1 + lambda_factor * df['NormalizedDemand'])
df['Price_Model2'] = df['Price_Model2'].clip(lower=0.5 * BASE_PRICE, upper=2.0 * BASE_PRICE)
df['Price_Model2'] = df['Price_Model2'].round(2)

# Final output preview
df[['SystemCodeNumber', 'Timestamp', 'Occupancy', 'QueueLength', 'VehicleType', 'TrafficConditionNearby',
    'IsSpecialDay', 'Price_Model2']].head(10)


,SystemCodeNumber,Timestamp,Occupancy,QueueLength,VehicleType,TrafficConditionNearby,IsSpecialDay,Price_Model2
0,BHMBCCMKT01,2016-10-04 07:59:00,61,1,car,low,0,11.61
1,BHMBCCMKT01,2016-10-04 08:25:00,64,1,car,low,0,11.62
2,BHMBCCMKT01,2016-10-04 08:59:00,80,2,car,low,0,12.62
3,BHMBCCMKT01,2016-10-04 09:32:00,107,2,car,low,0,12.67
4,BHMBCCMKT01,2016-10-04 09:59:00,150,2,bike,low,0,12.22
5,BHMBCCMKT01,2016-10-04 10:26:00,177,3,car,low,0,13.78
6,BHMBCCMKT01,2016-10-04 10:59:00,219,6,truck,high,0,16.73
7,BHMBCCMKT01,2016-10-04 11:25:00,247,5,car,average,0,15.64
8,BHMBCCMKT01,2016-10-04 11:59:00,259,5,cycle,average,0,14.91
9,BHMBCCMKT01,2016-10-04 12:29:00,266,8,bike,high,0,17.67


# 🛠️ Code: Model 3 – Competitor-Based Pricing

In [13]:
from sklearn.neighbors import NearestNeighbors

# Step 1: Prepare Data
df_competitor = df.copy()
df_competitor['Latitude'] = df_competitor['Latitude'].astype(float)
df_competitor['Longitude'] = df_competitor['Longitude'].astype(float)

# Step 2: Fit Nearest Neighbors on unique parking lots
lots = df_competitor.groupby('SystemCodeNumber')[['Latitude', 'Longitude']].mean().reset_index()
neighbors_model = NearestNeighbors(n_neighbors=3, metric='euclidean')
neighbors_model.fit(lots[['Latitude', 'Longitude']])
distances, indices = neighbors_model.kneighbors(lots[['Latitude', 'Longitude']])

# Step 3: Create mapping of each lot to its neighbors
neighbor_map = dict()
for i, lot in enumerate(lots['SystemCodeNumber']):
    neighbor_lots = lots.iloc[indices[i]]['SystemCodeNumber'].tolist()
    neighbor_lots.remove(lot)  # remove itself
    neighbor_map[lot] = neighbor_lots

# Step 4: Apply Model 3 logic
def compute_model3_price(row):
    lot = row['SystemCodeNumber']
    timestamp = row['Timestamp']

    # Get neighbor lots
    neighbors = neighbor_map.get(lot, [])

    # Fetch neighbor prices at same time
    competitors = df_competitor[(df_competitor['SystemCodeNumber'].isin(neighbors)) &
                                (df_competitor['Timestamp'] == timestamp)]

    if competitors.empty:
        return row['Price_Model2']  # fallback

    avg_competitor_price = competitors['Price_Model2'].mean()

    if row['Price_Model2'] < avg_competitor_price:
        return min(row['Price_Model2'] * 1.05, 2.0 * BASE_PRICE)  # increase slightly
    elif row['Price_Model2'] > avg_competitor_price:
        return max(row['Price_Model2'] * 0.95, 0.5 * BASE_PRICE)  # decrease slightly
    else:
        return row['Price_Model2']  # keep same

# Step 5: Apply to dataset
df['Price_Model3'] = df.apply(compute_model3_price, axis=1)
df['Price_Model3'] = df['Price_Model3'].round(2)


In [14]:
df[['SystemCodeNumber', 'Timestamp', 'Price_Model1', 'Price_Model2', 'Price_Model3']].head(10)


,SystemCodeNumber,Timestamp,Price_Model1,Price_Model2,Price_Model3
0,BHMBCCMKT01,2016-10-04 07:59:00,10.00,11.61,12.19
1,BHMBCCMKT01,2016-10-04 08:25:00,10.21,11.62,11.04
2,BHMBCCMKT01,2016-10-04 08:59:00,10.43,12.62,11.99
3,BHMBCCMKT01,2016-10-04 09:32:00,10.71,12.67,12.04
4,BHMBCCMKT01,2016-10-04 09:59:00,11.08,12.22,11.61
5,BHMBCCMKT01,2016-10-04 10:26:00,11.60,13.78,13.09
6,BHMBCCMKT01,2016-10-04 10:59:00,12.21,16.73,15.89
7,BHMBCCMKT01,2016-10-04 11:25:00,12.97,15.64,14.86
8,BHMBCCMKT01,2016-10-04 11:59:00,13.83,14.91,15.66
9,BHMBCCMKT01,2016-10-04 12:29:00,14.73,17.67,18.55


# Step 6: Visualizing Price Trends using Bokeh

Code: Line Chart in Bokeh

In [18]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import Legend
from bokeh.palettes import Category10
from bokeh.layouts import column

output_notebook()

# Convert timestamp to datetime if needed
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Create Bokeh figure
p = figure(x_axis_type="datetime", width=800, height=400, title="Parking Price Trends (Model 1 vs 2 vs 3)")
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Price (₹)'

# Line plots
colors = Category10[3]
line1 = p.line(df['Timestamp'], df['Price_Model1'], line_color=colors[0], line_width=2, legend_label="Model 1: Occupancy-Based")
line2 = p.line(df['Timestamp'], df['Price_Model2'], line_color=colors[1], line_width=2, legend_label="Model 2: Demand-Based")
p.line(df['Timestamp'], df['Price_Model3'], line_color="#2ca02c", line_width=2, legend_label="Model 3: Competitor-Based")

# Style
p.legend.location = "top_left"
p.legend.click_policy = "hide"
p.grid.grid_line_alpha = 0.3

show(p)


In [28]:
%whos DataFrame


Variable        Type         Data/Info
--------------------------------------
df              DataFrame          SystemCodeNumber  C<...>[18368 rows x 17 columns]
df_competitor   DataFrame          SystemCodeNumber  C<...>[18368 rows x 16 columns]
group           DataFrame          SystemCodeNumber  C<...>n[1312 rows x 11 columns]
lots            DataFrame        SystemCodeNumber   La<...>ing  26.150504  91.733531


In [29]:
df.columns


Index(['SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude', 'Occupancy',
       'VehicleType', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay',
       'Timestamp', 'Price_Model1', 'VehicleWeight', 'TrafficLevel',
       'RawDemand', 'NormalizedDemand', 'Price_Model2', 'Price_Model3'],
      dtype='object')

# Save the Final Output

In [30]:
df.to_csv("dynamic_pricing_output.csv", index=False)
print("✅ File saved successfully as dynamic_pricing_output.csv")


✅ File saved successfully as dynamic_pricing_output.csv
